In [62]:
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

In [63]:
%run TableCreationBaseline.py
%run addMidruleToLatex.py
%run mapRunNames.py

In [64]:
year = 2018

In [65]:
bltable = getBaselineDf()
blline = bltable.query("year == " + str(year)).drop(["P_10", "year"], axis=1).stack()
blline.index = blline.index.droplevel()

In [66]:
allstats=[]

In [67]:
def printBestStats(run):
    df = pd.read_csv("/Users/faessler/nfshome/Research/sigir/boostopt/"+run+"_analysis"+str(year)+".csv").round(2)
    df["year"] = pd.Series([year]*len(df), index=df.index)
    bestidx = df["infNDCG"].idxmax()
    best = df.loc[bestidx]
    print(best)
    difference = blline["infNDCG"] - best["infNDCG"]
    print("Difference to original baseline:", difference)
    meanrank = best["mean_r"]
    std      = best["std_r"]
    recall   = best["recall"]
    bestparams = df[list(filter(lambda x: "boost" in x, df.columns))].loc[bestidx]
    allstats.append( {"run":run, "bestscore": best["infNDCG"], "difftobaseline": difference, "bestparams":bestparams.to_dict(), "mean_r": meanrank, "std_r": std, "recall": recall})

# Positive and negative PM boosters

In [68]:
printBestStats("posneg")

posbooster       0.50
negbooster       0.00
recall          93.47
relgs         5588.00
relfound      5223.00
ndcg            52.14
infNDCG         37.01
Rprec           25.06
P_10            42.40
mean_r        1131.45
std_r         1717.41
year          2018.00
Name: 6, dtype: float64
Difference to original baseline: 16.64


# Additional terms

In [69]:
printBestStats("additional")

expansions     DSY_DHY_DP_GSY_GDE_DGI
cancerboost                       0.5
chemoboost                        1.5
dnaboost                          0.5
recall                          93.15
relgs                            5588
relfound                         5205
ndcg                            47.72
infNDCG                         31.86
Rprec                           21.84
P_10                             33.4
mean_r                        1268.66
std_r                         1804.24
year                             2018
Name: 10, dtype: object
Difference to original baseline: 21.79


# Fields

In [70]:
printBestStats("fields")

template          baseline
titleboost             2.5
abstractboost            2
keywordboost             1
meshboost                1
genefieldboost           1
recall               93.74
relgs                 5588
relfound              5238
ndcg                 59.97
infNDCG              47.82
Rprec                32.31
P_10                  53.4
mean_r              841.58
std_r              1483.62
year                  2018
Name: 621, dtype: object
Difference to original baseline: 5.829999999999998


# Mutations

In [71]:
printBestStats("mutation")

mutationboost       1.00
recall             93.20
relgs            5588.00
relfound         5208.00
ndcg               49.31
infNDCG            33.70
Rprec              23.36
P_10               34.80
mean_r           1200.68
std_r            1779.44
year             2018.00
Name: 1, dtype: float64
Difference to original baseline: 19.949999999999996


# DGInt

In [72]:
printBestStats("drug")

drugboost       0.50
recall         93.04
relgs        5588.00
relfound     5199.00
ndcg           48.74
infNDCG        33.33
Rprec          23.33
P_10           34.60
mean_r       1184.17
std_r        1758.67
year         2018.00
Name: 0, dtype: float64
Difference to original baseline: 20.32


# Genes and Diseases

In [73]:
printBestStats("genedis")

geneboost           1.50
genedescboost       1.00
genesynboost        1.00
disboost            2.00
dissynboost         1.00
recall             93.09
relgs            5588.00
relfound         5202.00
ndcg               59.66
infNDCG            47.57
Rprec              31.58
P_10               51.00
mean_r            785.97
std_r            1351.60
year             2018.00
Name: 87, dtype: float64
Difference to original baseline: 6.079999999999998


# PM Classifier

In [74]:
df = pd.read_csv("/Users/faessler/nfshome/Research/sigir/boostopt/pmclassboostopt_analysis"+str(year)+".csv").round(2)
df["year"] = pd.Series([year]*len(df), index=df.index)
# Exclude classifiers trained on 2018 data, that is unfair
df = df.query("not classifier.str.contains('"+str(year)+"')")
bestidx = df["infNDCG"].idxmax()
best = df.loc[bestidx]
print(best)
difference =  blline["infNDCG"]-best["infNDCG"]
print("Difference to original baseline:",difference )
meanrank = best["mean_r"]
std      = best["std_r"]
recall   = best["recall"]
bestparams = df[list(filter(lambda x: "boost" in x, df.columns))].loc[bestidx]
allstats.append( {"run":"pmclassboostopt", "bestscore": best["infNDCG"], "difftobaseline": difference, "bestparams":bestparams.to_dict(), "mean_r": meanrank, "std_r": std, "recall": recall})

template                 baseline_should
classboost                             1
classifier    pmclass2017lstmgru.keyword
recall                             93.84
relgs                               5588
relfound                            5244
ndcg                               60.94
infNDCG                             48.6
Rprec                               32.8
P_10                                53.2
mean_r                            780.62
std_r                            1392.08
year                                2018
Name: 77, dtype: object
Difference to original baseline: 5.049999999999997


In [75]:
table = pd.DataFrame(allstats)
table = table[["run", "recall", "bestscore", "difftobaseline", "mean_r", "std_r"]]
print(table.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
             run &  recall &  bestscore &  difftobaseline &   mean\_r &    std\_r \\
\midrule
          posneg &   93.47 &      37.01 &           16.64 &  1131.45 &  1717.41 \\
      additional &   93.15 &      31.86 &           21.79 &  1268.66 &  1804.24 \\
          fields &   93.74 &      47.82 &            5.83 &   841.58 &  1483.62 \\
        mutation &   93.20 &      33.70 &           19.95 &  1200.68 &  1779.44 \\
            drug &   93.04 &      33.33 &           20.32 &  1184.17 &  1758.67 \\
         genedis &   93.09 &      47.57 &            6.08 &   785.97 &  1351.60 \\
 pmclassboostopt &   93.84 &      48.60 &            5.05 &   780.62 &  1392.08 \\
\bottomrule
\end{tabular}



In [76]:
allstats

[{'bestparams': {'negbooster': 0.0, 'posbooster': 0.5},
  'bestscore': 37.01,
  'difftobaseline': 16.64,
  'mean_r': 1131.45,
  'recall': 93.47,
  'run': 'posneg',
  'std_r': 1717.41},
 {'bestparams': {'cancerboost': 0.5, 'chemoboost': 1.5, 'dnaboost': 0.5},
  'bestscore': 31.86,
  'difftobaseline': 21.79,
  'mean_r': 1268.66,
  'recall': 93.15,
  'run': 'additional',
  'std_r': 1804.24},
 {'bestparams': {'abstractboost': 2.0,
   'genefieldboost': 1.0,
   'keywordboost': 1.0,
   'meshboost': 1.0,
   'titleboost': 2.5},
  'bestscore': 47.82,
  'difftobaseline': 5.829999999999998,
  'mean_r': 841.58,
  'recall': 93.74,
  'run': 'fields',
  'std_r': 1483.62},
 {'bestparams': {'mutationboost': 1.0},
  'bestscore': 33.7,
  'difftobaseline': 19.949999999999996,
  'mean_r': 1200.68,
  'recall': 93.2,
  'run': 'mutation',
  'std_r': 1779.44},
 {'bestparams': {'drugboost': 0.5},
  'bestscore': 33.33,
  'difftobaseline': 20.32,
  'mean_r': 1184.17,
  'recall': 93.04,
  'run': 'drug',
  'std_r': 